# Import required files

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import inflect
from num2words import num2words 
import re

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [74]:
#from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))
p = inflect.engine()

# Cardinal class conversion

In [75]:
def cardinal(x):
    try:
        if re.match('.*V|X|I|L|D',x):            
            x = rom_to_int(x)
            return num2words(x)
        if re.match('.*[A-Za-z]+.*', x):
            return x        
        x = re.sub(',', '', x, count = 10)

        if(re.match('.+\..*', x)):
            x = p.number_to_words(float(x))
        elif re.match('\..*', x): 
            x = p.number_to_words(float(x))
            x = x.replace('zero ', '', 1)
        else:
            x = p.number_to_words(int(x))
        x = x.replace('zero', 'o')    
        x = re.sub('-', ' ', x, count=10)
        x = re.sub(' and','',x, count = 10)
        x = re.sub(',', '', x, count = 10)
        return x
    except:
        return x

# Digit converter

In [76]:
def digit(x): 
    try:
        x = re.sub('[^0-9]', '',x)
        result_string = ''
        for i in x:
            result_string = result_string + cardinal(i) + ' '
        result_string = result_string.strip()
        return result_string
    except:
        return(x) 

# Letter conversion function

In [106]:
def letters(x):
    try:
        x = re.sub('[^a-zA-Z]', '', x)
        x = x.lower()
        result_string = ''
        for i in range(len(x)):
            result_string = result_string + x[i] + ' '
        return(result_string.strip())  
    except:
        return x

# Ordinal class

In [60]:
def rom_to_int(string):

    table=[['M',1000],['CM',900],['D',500],['CD',400],['C',100],['XC',90],['L',50],['XL',40],['X',10],['IX',9],['V',5],['IV',4],['I',1]]
    returnint=0
    for pair in table:


        continueyes=True

        while continueyes:
            if len(string)>=len(pair[0]):

                if string[0:len(pair[0])]==pair[0]:
                    returnint+=pair[1]
                    string=string[len(pair[0]):]

                else: continueyes=False
            else: continueyes=False

    return returnint    
def ordinal(x):
    try:
        result_string = ''
        x = x.replace(',', '')
        x = x.replace('[\.]$', '')
        if re.match('^[0-9]+$',x):
            x = num2words(int(x), ordinal=True)
            return(x.replace('-', ' '))
        if re.match('.*V|X|I|L|D',x):
            if re.match('.*th|st|nd|rd',x):
                x = x[0:len(x)-2]
                x = rom_to_int(x)
                result_string = re.sub('-', ' ',  num2words(x, ordinal=True))
            else:
                x = rom_to_int(x)
                result_string = 'the '+ re.sub('-', ' ',  num2words(x, ordinal=True))
        else:
            x = x[0:len(x)-2]
            result_string = re.sub('-', ' ',  num2words(float(x), ordinal=True))
        return(result_string)  
    except:
        return x

# Converter for Address

In [85]:
def address(x):
    try:
        x = re.sub('[^0-9a-zA-Z]+', '', x)
        result_string = ''
        for i in range(0,len(x)):
            if re.match('[A-Z]+',x[i]):                
                result_string = result_string + (x[i]).lower() + ' '                
            else:
                result_string = result_string + cardinal(x[i]) + ' '         
        return(result_string.strip())        
    except:    
        return(x)

# Telephone class prediction

In [87]:
def telephone(x):
    try:
        result_string = ''
        
        for i in range(0,len(x)):
            if re.match('[0-9]+', x[i]):
                result_string = result_string + cardinal(x[i]) + ' '
            else:
                result_string = result_string + 'sil '
        return result_string.strip()    
    except:    
        return(x)

# Electronic class converter

In [89]:
def electronic(x):
    try:
        replacement = {'.' : 'dot', ':' : 'colon', '/':'slash', '-' : 'dash', '#' : 'hash tag', }
        result_string = ''
        if re.match('.*[A-Za-z].*', x):
            for char in x:
                if re.match('[A-Za-z]', char):
                    result_string = result_string + letters(char) + ' '
                elif char in replacement:
                    result_string = result_string + replacement[char] + ' '
                elif re.match('[0-9]', char):
                    if char == 0:
                        result_string = result_string + 'o '
                    else:
                        number = cardinal(char)
                        for n in number:
                            result_string = result_string + n + ' ' 
            return result_string.strip()                
        else:
            return(x)
    except:    
        return(x)

# Fraction class converter

In [112]:
def fraction(x):
    try:
        y = x.split('/')        
        result_string = ''
        y[0] = cardinal(y[0].strip())        
        y[1] = ordinal(y[1].strip())
        if y[1] == 4:
            result_string = y[0] + ' quarters'
        else:    
            result_string = y[0] + ' ' + y[1] + 's'
        return(result_string)
    except:    
        return(x)

# Converter for Money class

In [116]:
def money(x):
    try:
        if re.match('^\$', x):
            x = x.replace('$','')
            if len(x.split(' ')) == 1:
                if re.match('.*M|m$',x):
                    x = x.replace('M', '')
                    x = x.replace('m', '')
                    text = cardinal(x)
                    x = text + ' million dollars'
                elif re.match('.*b|B$', x):
                    x = x.replace('B', '')
                    x = x.replace('b', '')
                    text = cardinal(x)
                    x = text + ' million dollars'
                else:
                    text = cardinal(x)
                    x = text + ' dollars'
                return x.lower()
            elif len(x.split(' ')) == 2:
                text = cardinal(x.split(' ')[0])
                if x.split(' ')[1].lower() == 'million':
                    x = text + ' million dollars'
                elif x.split(' ')[1].lower() == 'billion':
                    x = text + ' billion dollars'
                return x.lower()
                
                
                
        if re.match('^US\$', x):
            x = x.replace('US$','')
            if len(x.split(' ')) == 1:
                if re.match('.*M|m$', x):
                    x = x.replace('M', '')
                    x = x.replace('m', '')
                    text = cardinal(x)
                    x = text + ' million dollars'
                elif re.match('.*b|B$', x):
                    x = x.replace('b', '')
                    x = x.replace('B', '')
                    text = cardinal(x)
                    x = text + ' million dollars'
                else:
                    text = cardinal(x)
                    x = text + ' dollars'
                return x.lower()
            elif len(x.split(' ')) == 2:
                text = cardinal(x.split(' ')[0])
                if x.split(' ')[1].lower() == 'million':
                    x = text + ' million dollars'
                elif x.split(' ')[1].lower() == 'billion':
                    x = text + ' billion dollars'
                return x.lower()

        elif re.match('^£', x):
            x = x.replace('£','')
            if len(x.split(' ')) == 1:
                if re.match('.*M|m$', x):
                    x = x.replace('M', '')
                    x = x.replace('m', '')
                    text = cardinal(x)
                    x = text + ' million pounds'
                elif re.match('.*b|B$', x):
                    x = x.replace('b', '')
                    x = x.replace('B', '')
                    text = cardinal(x)
                    x = text + ' million pounds'
                else:
                    text = cardinal(x)
                    x = text + ' pounds'
                return x.lower()
            elif len(x.split(' ')) == 2:
                text = cardinal(x.split(' ')[0])
                if x.split(' ')[1].lower() == 'million':
                    x = text + ' million pounds'
                elif x.split(' ')[1].lower() == 'billion':
                    x = text + ' billion pounds'
                return x.lower()
            
        elif re.match('^€', x):
            x = x.replace('€','')
            if len(x.split(' ')) == 1:
                if re.match('.*M|m$', x):
                    x = x.replace('M', '')
                    x = x.replace('m', '')
                    text = cardinal(x)
                    x = text + ' million euros'
                elif re.match('.*b|B$', x):
                    x = x.replace('B', '')
                    x = x.replace('b', '')
                    text = cardinal(x)
                    x = text + ' million euros'
                else:
                    text = cardinal(x)
                    x = text + ' euros'
                return x.lower()
            elif len(x.split(' ')) == 2:
                text = cardinal(x.split(' ')[0])
                if x.split(' ')[1].lower() == 'million':
                    x = text + ' million euros'
                elif x.split(' ')[1].lower() == 'billion':
                    x = text + ' billion euros'
                return x.lower()  
    except:    
        return(x)

# Conversion for Plain class

In [4]:
data = pd.read_csv('./en_train.csv')

MemoryError: 

In [182]:
from tqdm import *

In [1]:
#def plain_array():
ln = len(data.loc[data['class'] == 'PLAIN'])
dt = data.loc[data['class'] == 'PLAIN']
before = np.array(dt['before'])
after = np.array(dt['after'])

different = {}

for i in tqdm(range(before.size)):    
    if(before[i] != after[i] and before[i] not in different):
        different[before[i]] = after[i]

    #return different

NameError: name 'data' is not defined

In [188]:
#np.save(file='plain_file',arr=different)

In [190]:
#different = np.load('./plain_file.npy')

In [184]:
def plain(x):
    try:
        
        if x in different:
            return different[x]
        else:
            return x
    except:
        return(x)

# Conversion for DATE class

In [191]:
#dt = data.loc[data['class'] == 'MEASURE']
#print(dt[:50])

In [192]:
"""
from string import digits
unq_measures = []

for i in dt[0]:
    print(i)
    s = dt['before'][i]    
    remove_digits = str.maketrans('', '', digits)
    res = (s.translate(remove_digits)).strip()
    res = re.sub(',', '', res, count = 10)
    #res = re.sub(r'[^\w\s\%]', '', res, count = 10)
    #res = re.sub('/', '', res, count = 10)
    res = res.strip()
    res += ' ' + dt['after'][i]
    if(res not in unq_measures):
        unq_measures.append(res)
"""

"\nfrom string import digits\nunq_measures = []\n\nfor i in dt[0]:\n    print(i)\n    s = dt['before'][i]    \n    remove_digits = str.maketrans('', '', digits)\n    res = (s.translate(remove_digits)).strip()\n    res = re.sub(',', '', res, count = 10)\n    #res = re.sub(r'[^\\w\\s\\%]', '', res, count = 10)\n    #res = re.sub('/', '', res, count = 10)\n    res = res.strip()\n    res += ' ' + dt['after'][i]\n    if(res not in unq_measures):\n        unq_measures.append(res)\n"

In [190]:
def date(x):
    result = ""
    month = {'1' : 'january', '2' : 'february', '3':'march', '4' : 'april', '5' : 'may', '6' : 'june', '7' : 'july', '8' : 'august', '9' : 'september', '10' : 'october', '11' : 'november', '12' : 'december'}
    if(len(x) == 4):
        if(int(x) < 2000 or int(x) >= 2010):            
            x = p.number_to_words(int(x), group = 2)
        else:
            x = p.number_to_words(int(x))
    elif(re.search(r'(?:\d{1,2} )?(?: Empty|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.]?[-]? (?:\d{1,2}[a-z]*[,]? )?[-]?\d{4}', x)):        
        tuples = re.findall(r'((?:\d{1,2} )?)((?: Empty|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[.]?[-]? ((?:\d{1,2}[a-z]*[,]? )?)[-]?(\d{4})', x)        
        if(tuples[0][0] != ''):
            date = tuples[0][0].strip()            
            result += str('the ' + ordinal(date) + ' of ')        
        if(tuples[0][1] != ''):
            result += str(tuples[0][1].lower()) + ' '
        if(tuples[0][2] != ''):
            date = tuples[0][2].strip()
            date = re.sub(',', '', date, count = 10)            
            result += str(ordinal(date) + ' ')
        if(tuples[0][3] != ''):            
            if(int(tuples[0][3]) < 2000 or int(tuples[0][3]) >= 2010):            
                result += str(p.number_to_words(int(tuples[0][3]), group = 2))
            else:
                result += str(p.number_to_words(int(tuples[0][3])))
        x = result
    elif(re.search(r'\d{1,2}[/]\d{1,2}[/]\d{2,4}', x)):            
        tuples = re.findall(r'(\d{1,2})[/](\d{1,2})[/](\d{2,4})', x)
        if(tuples[0][0] != ''):
            result += month[tuples[0][0]] + ' '
        if(tuples[0][1] != ''):
            result += ordinal(tuples[0][1]) + ' '
        if(tuples[0][2] != ''):
            result += digit(tuples[0][2])   
        x = result
    elif(re.search(r'(?: Empty|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.]?[-]? (\d{2})', x)):
        tuples = re.findall(r'((?: Empty|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.]?[-]?) (\d{2})', x)
        print(tuples)
        if(tuples[0][0] != ''):
            result += str(tuples[0][0].lower()) + ' '
        if(tuples[0][1] != ''):
            date = tuples[0][1].strip()                        
            result += str(ordinal(date))
        x = result
    elif(re.search(r'(?:\d{1,2} )?(?: Empty|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.]?[-]?', x)):
        print('Here')
        tuples = re.findall(r'((?:\d{1,2} )?)((?: Empty|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)', x)
        print(tuples)
        if(tuples[0][0] != ''):
            date = tuples[0][0].strip()
            result += 'the '+ str(ordinal(date)) + ' of '
        if(tuples[0][1] != ''):
            result += str(tuples[0][1].lower())
        x = result
    elif(re.search(r'\d{1,2}[-]\d{1,2}[-]\d{4}', x)):        
        tuples = re.findall(r'(\d{1,2})[-](\d{1,2})[-](\d{4})', x)        
        if(tuples[0][0] != ''):
            date = tuples[0][0].strip()
            result += 'the '+ str(ordinal(date)) + ' of '
        if(tuples[0][1] != ''):
            result += month[str(int(tuples[0][1]))] + ' '
        if(tuples[0][2] != ''):            
            if(int(tuples[0][2]) < 2000 or int(tuples[0][2]) >= 2010):            
                result += str(p.number_to_words(int(tuples[0][2]), group = 2))
            else:
                result += str(p.number_to_words(int(tuples[0][2])))
        x = result
    elif(re.search(r'\d{4}[-]\d{1,2}[-]\d{1,2}', x)):        
        tuples = re.findall(r'(\d{4})[-](\d{1,2})[-](\d{1,2})', x)        
        if(tuples[0][2] != ''):
            date = tuples[0][2].strip()
            result += 'the '+ str(ordinal(date)) + ' of '
        if(tuples[0][1] != ''):
            result += month[str(int(tuples[0][1]))] + ' '
        if(tuples[0][0] != ''):            
            if(int(tuples[0][0]) < 2000 or int(tuples[0][0]) >= 2010):            
                result += str(p.number_to_words(int(tuples[0][0]), group = 2))
            else:
                result += str(p.number_to_words(int(tuples[0][0])))
        x = result
    x = x.replace('zero', 'o')    
    x = re.sub('-', ' ', x, count=10)
    x = re.sub(' and','',x, count = 10)
    x = re.sub(',', '', x, count = 10)
    return x

# Conversion for Verbatim

In [199]:
from tqdm import *

In [193]:
ln = len(data.loc[data['class'] == 'VERBATIM'])
dt = data.loc[data['class'] == 'VERBATIM']
before = np.array(dt['before'])
after = np.array(dt['after'])

verbatim_dict = {}

for i in tqdm(range(before.size)):    
    if(before[i] != after[i] and before[i] not in verbatim_dict):
        verbatim_dict[before[i]] = after[i]
print(len(verbatim_dict))

100%|███████████████████████████████████████████████████████████████████████| 78108/78108 [00:00<00:00, 1599526.88it/s]


1365


In [196]:
def verbatim(x):
    try:
        if x in verbatim_dict:
            return verbatim_dict[x]
        else:
            return x
    except:
        return(x)

# Converter for Time

In [155]:
def time(x):
    result = ''
    if re.match(r'\d{1,2}[:]?[.]?(\d{1,2})?[:]?[\s]?(\d{1,2})?[A-za-z.\s]*',x):
        tuples = re.findall(r'(\d{1,2})[:]?[.]?(\d{1,2})?[:]?[\s]?([A-za-z.\s]*)(\d{1,2})?', x)
        print(tuples)
        if(tuples[0][0] != ''):
            result += cardinal(tuples[0][0]) + ' '
            if(tuples[0][3] != ''):
                result += ' ' + 'hours '
        if(tuples[0][1] != ''):            
            if(tuples[0][1] == '00'):
                result += ' ' + "o' clock"
            else:
                result += cardinal(tuples[0][1])
            if(tuples[0][3] != ''):
                result += ' ' + 'minutes '
        if(tuples[0][2] != ''):
            result += ' ' + letters(tuples[0][2])
        if(tuples[0][3] != ''):
            result += ' ' + cardinal(tuples[0][3]) + ' seconds'
        
    print(result)